# 6.2 Computing Word Similarities with PMI and SVD

The following is inspired by the blog post [simple word vectors with co-occurrence pmi and svd](https://www.kaggle.com/alexklibisz/simple-word-vectors-with-co-occurrence-pmi-and-svd) by Alex Klibisz.

In [1]:
from __future__ import print_function, division
from collections import Counter
from itertools import combinations
from math import log
from pprint import pformat
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds
from sklearn import preprocessing
from nltk import ngrams
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
import pickle
from tqdm.auto import tqdm
from ipywidgets import interact
import ipywidgets as widgets
import pandas as pd

First read in the list of list that was created in the previous notebook.

In [2]:
with open("output/data_for_pmi.p", "rb") as r:
    lemm_l = pickle.load(r)

In order to compute PMI we need a sliding window going over the data. There are various ways of creating such windows. The approach taken here is to use the `ngrams` function from the `nltk` package.
```python
from nltk import ngrams
n = 7
windows = ngrams(text, n)
```
This will create windows of the size `n`. Increasing `n` will increase the number of collocates and the time it takes to produce them. Thne minimum size of `n` is 2, which will only consider words that are immediately adjacent. Larger window sizes may capture the general theme of the passage from which the window is taken.

For each window the function `combinations()` (from the `itertool` library) is used to produce all possible combinations of 2 words in that window. With a 7-word window length, a sequence such as en-lil₂ kur gal-ra šulgi lugal-e e₂ mu-na-du₃ ("Šulgi the king build a temple for Great Mountain Enlil") will yield the following 21 word pairs as tuples:  
* (Enlil\[1\]DN, kur\[mountain\]N)
* (Enlil\[1\]DN, gal\[big\]V/i)
* (Enlil\[1\]DN, Šulgi\[1\]RN)
* (Enlil\[1\]DN, lugal\[king\]N)
* (Enlil\[1\]DN, e\[house\]N)
* (Enlil\[1\]DN, du\[build\]V/t)
* (kur\[mountain\]N, gal\[big\]V/i)
* (kur\[mountain\]N, Šulgi\[1\]RN)
* (kur\[mountain\]N, lugal\[king\]N)
* (kur\[mountain\]N, e\[house\]N)
* (kur\[mountain\]N, du\[build\]V/t)
* (gal\[big\]V/i, Šulgi\[1\]RN)
* (gal\[big\]V/i, lugal\[king\]N)
* (gal\[big\]V/i, e\[house\]N)
* (gal\[big\]V/i, du\[build\]V/t)
* (Šulgi\[1\]RN, lugal\[king\]N)
* (Šulgi\[1\]RN, e\[house\]N)
* (Šulgi\[1\]RN, du\[build\]V/t)
* (lugal\[king\]N, e\[house\]N)
* (lugal\[king\]N, du\[build\V/t])
* (e\[house\]N, du\[build\]V/t)
Each of these word pairs is considered a collocate. By reducing the window size one may concentrate on words that occur closer together, and may be part of a more or less fixed combination (such as kur gal "great mountain"as an epithet of Enlil). Larger window sizes will consider broader thematic issues, recognizing words that are used when discussing particular themes or issues.

The `Counter()` function (from the `collections` library) is used to establish the counts of unique words (variable `cx`) as well as the counts of unique collocates (variable `cxy`). The `Counter()` is first initialized as an empty variable and then updated while iterating over the list of lists `lemm_l`.

In [3]:
cx = Counter()
cxy = Counter()
windowsize = widgets.IntSlider(min=2, max=25, value=7, description="window size")
windowsize

IntSlider(value=7, description='window size', max=25, min=2)

In [4]:
for text in tqdm(lemm_l):
    cx.update(text)
    windows = ngrams(text, windowsize.value)
    for w in windows:
        z = [tuple(l) for l in map(sorted, combinations(w, 2))]
        cxy.update(z)  # count all collocates (word pairs)

## Removing stop words and rare words
# TODO: edit the text below and make decisions about stop word list
The following cell is used to remove very frequent and/or very rare lemmas, based on `min_count` and `max_count` thresholds. The variable `min_count` is currently set to 15. For low-frequency words the current approach is not reliable; the best setting of this parameter is still subject to further research and may well depend on the corpus that is being used for analysis. The variable `max_count` currently equals the total number of tokens in the corpus, which means that no [stop words](https://en.wikipedia.org/wiki/Stop_word) are discarded. For Sumerian, it is not obvious which words should count as stop words since Sumerian barely uses pronouns (where used they are highly significant) and has few true function words. For the Ur III corpus, one could designate words for units such as **gur**, **giŋ**, and **sila**, or time units such as **ud** (day) and **itud** (month) as stop words, since they add little indeed to the content of the documents. However, the other hand, the association of **gur**, **sila**, etc. with some commodities (counted by volume) and not with others (counted by piece or weight, etc.) is exactly the kind of information that is captured well with PMI. (For an excellent discussion of stop words see Julia Silge's [Supervised Machine Learning for Text Analysis in R](https://smltar.com/stopwords.html)).

==================

(one might argue that **ud** is in many cases a function word meaning "when"). (Add here **ki** (place) **mu** (name), **ŋiri** (foot) and **šuniŋin** (total); perhaps **kišib** (seal) and **dumu** (child)). Very frequent items such as **udu** sheep, **še** (barley), **ninda** (bread) or **lugal** (king) should not be removed, because those are the actual subjects of these texts.

=====================

Importantly, we do remove all underscores here. In our data, underscores are place holders that represent unlemmatized words (broken or unknown). They have been included in the process so far for the benefit of the sliding window in the previous section. Our list of collocates, therefore, has many entries such as (lugal\[king\]N, _ ) meaning that there are 7-word windows in which the word for king co-occurs with an unlemmatized word. These collocates are removed in the next cell.

In [5]:
print('%d unique tokens before' % len(cx))
print('%d total of tokens before' % sum(cx.values()))
sx = sum(cx.values())
min_count = 15
max_count = sx
for x in list(cx.keys()):
    if cx[x] < min_count or cx[x] > max_count:
        del cx[x]
del cx['_']  # underscores are place holders and may be removed now.
print('%d unique tokens after' % len(cx))
print('%d total of tokens after' % sum(cx.values()))
print('Most common:', cx.most_common()[:25])

28551 unique tokens before
2243562 total of tokens before
3709 unique tokens after
2033101 total of tokens after
Most common: [('sila[unit]N', 96554), ('giŋ[unit]N', 58899), ('itud[moon]N', 52132), ('dumu[child]N', 46623), ('gur[unit]N', 45371), ('ki[place]N', 44639), ('ud[sun]N', 43274), ('udu[sheep]N', 35534), ('še[barley]N', 34354), ('kaš[beer]N', 33902), ('šuniŋin[total]N', 32175), ('kišib[seal]N', 28136), ('ninda[bread]N', 27325), ('lugal[king]N', 24668), ('i[oil]N', 22847), ('gud[ox]N', 22401), ('ŋuruš[male]N', 21326), ('dubsar[scribe]N', 20452), ('šu[hand]N', 18033), ('niga[fattened]V/i', 17619), ('ugula[overseer]N', 16575), ('ŋiri[foot]N', 16113), ('sila[lamb]N', 16071), ('naŋa[potash]N', 15682), ('šum[garlic]N', 15479)]


# Clean the Collocates
Remove all bigrams that contain a lemma that has been removed in the previous step.

In [6]:
for x, y in list(cxy.keys()):
    if x not in cx or y not in cx:
        del cxy[(x, y)]

# Lookup Dictionaries
The following step creates two dictionaries. The dictionary `x2i` has the lemmas as keys and a counter as value. The dictionary `i2x` has that same counter as keys, and the lemmas as values. The counters will be used as indexes for the columns and rows of the matrix constructed below.

In [7]:
x2i, i2x = {}, {}
for i, x in enumerate(cx.keys()):
    x2i[x] = i
    i2x[i] = x

# Compute Token and Collocate Totals

In [8]:
sx = sum(cx.values())
sxy = sum(cxy.values())

# Create PMI matrix

In the next cell the data is transformed into a (sparse) matrix, with rows and columns representing unique words, and the data in each cell representing the PMI score for the co-occurence for the two words. The variable `cxy` contains a list of all collocates in the format `'Unug[1]SN', 'šag[heart]N'): 1653`; meaning that the collocation of these two terms appears 1653 times. The dictionary `x2i` is used to translate each lemma into an index number and append the numbers to the lists `rows` and `cols`. The third element that the matrix function need is `data`. Instead of simply entering the frequency in `data` we enter the PPMI score. The formula for PMI is  $$log\frac{p(x,y)}{p(x)p(y)}$$; that is: the logarithm of the probability of encountering x *and* y, divided by the probability of x times the probability of y. The probabilities are computed by dividing the frequency of x and y by the total number of tokens and dividing the frequency of (x, y) by the total number of collocates.

One problem with PMI is that it favors low frequency terms. If term A only appears 10 times in our dataset, each collocate with any other term will appear significant, because it occurs in 10% of all the occurences of term A. We will see this effect, for instance, in the appearance of rare names. One solution is using PMI2, where the probability of the collocate is squared: PMI2 = $$log\frac{p(x,y)^2}{p(x)p(y)}$$

Still other possibility: PMIα: raise p(y) to the power of α, usually 0.75. PMIα = $$log\frac{p(x,y)}{p(x)p(y)^α}$$


In [9]:
flavors = ["PPMI", "PMI2", "PMIα"]
pmi_flavor = widgets.Dropdown(options=flavors, value="PMIα")
pmi_flavor

Dropdown(index=2, options=('PPMI', 'PMI2', 'PMIα'), value='PMIα')

In [10]:
pmi_samples = Counter()
data, rows, cols = [], [], []
for (x, y), n in tqdm(cxy.items()):
    rows.append(x2i[x])
    cols.append(x2i[y])
    if pmi_flavor.value == "PPMI":
        data.append(max(log((n / sxy) / (cx[x] / sx) / (cx[y] / sx)), 0)) # PPMI
    elif pmi_flavor.value == "PMI2":
        data.append(abs(log(((n / sxy)**2) / (cx[x] / sx) / (cx[y] / sx)))) # PMI2
    else:
        data.append(max(log((n / sxy) / (cx[x] / sx) / (cx[y] / sx)**.75), 0)) # PMIα 
    pmi_samples[(x, y)] = data[-1]
PMI = csc_matrix((data, (rows, cols)))
print('%d non-zero elements' % PMI.count_nonzero())
print('Sample values\n', pformat(pmi_samples.most_common(10)))


211279 non-zero elements
Sample values
 [(('suhuš[offshoot]N', 'suhuš[offshoot]N'), 7.863024544102512),
 (('UŠ.BAR[responsible]N', 'nam.DUN₃@g.la₂[unmng]N'), 7.702456040426489),
 (('izi[fire]N', 'la[stretch]V/t'), 7.532454212861611),
 (('Ad.da.ri₂[00]PN', 'Ši.ba.la[00]PN'), 7.466136890374496),
 (('esi[diorite]N', 'esi[diorite]N'), 7.409834462514443),
 (('Ad.mu[00]PN', 'Da.ri₂.ša[00]PN'), 7.339339966902755),
 (('dag[demolish]V/t', 'ŋala[cvne]N'), 7.320726804668015),
 (('Amar.suen.ma.lik[00]PN', 'DUN₃@g.la₂[shekel-weigher?]N'), 7.15512464030069),
 (('amsi[elephant]N', 'zu[tooth]N'), 7.111117795338952),
 (('Ad.da.ri₂[00]PN', 'Wa.mu[00]PN'), 6.958605517199234)]


# SVD
The sparse PMI matrix, which has as many rows and columns as there are unique lemmas, is factorized by using Single Value Decomposition. The number of columns is reduced by computing the first *k* factors that best explain the variance in the matrix. We end up with a vector for each word (a row in the new matrix) of length *k*.

SVD decomposes a matrix into three smaller matrices that, together, can reconstruct the original matrix. For our purposes, we only need the first of those. The underscores in the command line
```python
U, _, _ = svds(PPMI, k)
```
represent the other two matrices, which are discarded.

In [11]:
k = 50
U, _, _ = svds(PMI, k)

# Normalizing
The resulting vectors (the rows in the reduced matrix) are normalized, so that each vector has length 1 (L2 normalization). The `preprocessing.normalize()`, part of the `sklearn` package, does that job. Normalization is necessary for computing cosine similarity in the next cell.

In [12]:
U = preprocessing.normalize(U, norm='l2')

# Similarity
The similarity between two vectors, representing lemmas, is expressed in cosine similarity. Cosine similarity is the cosine of the angle between two vectors and ranges form 0 (no similarity at all; orthogonal vectors) to 1 (full similarity). For normalized vectors cosine similarity may be computed with the dot product of the two vectors. For our purpose, we need to compute the cosine similarity between our target word and all other words in our collection. The dot product of the SVD matrix with vector of the target word will provide the full collection of cosine similarity values. This set of values, called `dd` is in the same order (using the same indexes) as our original index, so that when `U[i]` (the *i-th* row of the matrix) represents the vector of **kugsig\[gold\]N**, `i2x[i]` = **kugsig\[gold\]N** and `dd[i]` represents the cosine similarity of **kugsig\[gold\]N** with the target word.

To find the highest scoring words in our vocabulary we use the `argsort()` function from the `numpy` library. This function will yield the indexes of a `numpy` array from the lowest value to the highest. Since we need the highest values, we use `[::-1]` to go through the array step by step backwards and select the last `k` indexes in the array. Among the highest scoring words will always be the target word itself (which should be represented by the value 1 in `dd`). Therefore, we will retrieve `k = n+1` values, while skipping the target word in the output.

In [13]:
def nearestwords(word, n=5):
    k = n + 1
    s = ''
    dd = np.dot(U, U[x2i[word]])
    for i in np.argsort(dd)[::-1][:k]:        # sort the array by value from low to high. Select the highest k values.
        if i2x[i] == word: continue           # omit the target word itself
        s += '(%s, %.3lf, %s) ' % (i2x[i], dd[i], cx[i2x[i]])
    print('PMI %s, %d\n %s' % (word, cx[word], s))
    print('-' * 10)

In [19]:
sortorder = " ʾ'[]aāâbcdeēêfgŋhiīîjklmnopqrsṣštṭuūûvwxyz0123456789₀₁₂₃₄₅₆₇₈₉ₓ{}().-/~?@×|&<>"
word = sorted(cx.keys(), key=lambda w: [sortorder.index(c.casefold()) for c in w]) # use custom sort order
interact(nearestwords, word = word, n = (1, 25));

interactive(children=(Dropdown(description='word', options=('aʾabak[sea]N', 'aʾaŋa[assignment]N', 'Aʾalimmah[0…

# Select output by POS

The following cells provide an interactive tool for analyzing the word vectors that were produced with PMI and SVD. The output is a Pandas DataFrame with nearby neighbors, that is, words that are represented by vectors that are relatively close to the vector of target word. One may restrict the output by Part of Speech and the output includes links to the [ePSD2](http://oracc.org/epsd2) pages for these words. The links to [ePSD2](http://oracc.org/epsd2) depend on the stable OID numbers (Oracc Identifiers). All Sumerian words have OIDs, but many proper nouns still have temporary Citation Forms and Guide Words and therefore have no OID. Such words/names do have dictionary pages, but those pages must be found by searching in [ePSD2/admin/ur3/qpn](http://oracc.org/epsd2/admin/ur3/qpn).

In [ ]:
anchor = '<a href="http://oracc.org/epsd2/{}", target="_blank">{}</a>'
columns = ["word", "sim.", "freq."]
with open("output/x2oid.p", "rb") as r:
    x2oid = pickle.load(r)

In [ ]:
def nearestwordsselect(words, POSfilter = [], n=5):
    n = n+len(words)
    l = []
    vecs = [U[x2i[word]] for word in words]
    vecm = np.mean(vecs, axis = 0)
    dd = np.dot(U, vecm)
    ds = np.argsort(dd)           # sort the indexes of the dd array
    if POSfilter: 
        F = [i for i in i2x.keys() if i2x[i].split(']')[1] in POSfilter]
        ds = ds[np.isin(ds, F)]       # select the indexes that appear in F
    #if len(words) == 1: 
    #    ds = ds[ds != x2i[words[0]]]      # omit target word
    for i in ds[::-1][:n]:        # Select the highest n values
        e = [i2x[i], dd[i], cx[i2x[i]]]
        if x2oid.get(e[0]): 
            e[0] = anchor.format(x2oid.get(e[0]), e[0])
        l.append(e)
    df = pd.DataFrame(l, columns=columns)
    print(words)
    return df.style.format({"sim." : '{0:,.3f}'})

In [ ]:
POS = ['N', 'V/i', 'V/t', 'AJ', 'PN', 'DN', 'SN', 'WN', 'FN', 'ON', 'TN', 'NU']
interact(nearestwordsselect, 
         words = widgets.SelectMultiple(options = word, 
                                  description = "Word", 
                                  value = [word[0]]),
         POSfilter=widgets.SelectMultiple(options = POS, 
                                  value = [], 
                                  description = "POS"), 
         n=(1,25)
                                      );

# Save Vectors in Gensim Keyed Vectors Format
The Gensim Keyed Vectors format has in the first row the dimensions of the data set (number of vectors and number of places in each vector). Each subsequent line begins with a key (the word), followed by the vector values, all separated by spaces.

In [ ]:
firstline = f"{str(len(U))} {str(len(U[0]))}\n"
vec = pd.DataFrame(U, index=x2i.keys())
with open("output/vec_file.txt", "w", encoding = "utf8") as w:
    w.write(firstline)
with open("output/vec_file.txt", "a", encoding="utf8") as w:
    vec.to_csv(w, sep=" ", header=False)